In [1]:
!pip install torch torchvision torchaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 4.6 MB/s eta 0:00:00


In [2]:
# Clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt  # Install YOLOv5 requirements

Cloning into 'yolov5'...
remote: Enumerating objects: 16525, done.
remote: Total 16525 (delta 0), reused 0 (delta 0), pack-reused 16525
Receiving objects: 100% (16525/16525), 15.00 MiB | 24.30 MiB/s, done.
Resolving deltas: 100% (11361/11361), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 749.1/749.1 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.6 MB/s eta 0:00:00


In [3]:
import torch

# Load the pre-trained YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)


/usr/local/lib/python3.10/dist-packages/torch/hub.py:294: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2024-4-4 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

100%|██████████| 14.1M/14.1M [00:00<00:00, 128MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [4]:
folder_path_drive = "pictures"
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
!pip install pytorch_lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 801.6/801.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 38.8 MB/s eta 0:00:00


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from PIL import Image
import os
import pytorch_lightning as pl
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

class InstagramImageDataset(Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image

class ImageCroppingModel(pl.LightningModule):
    def __init__(self, header_ratio=0.17, footer_ratio=0.39):
        super(ImageCroppingModel, self).__init__()
        self.header_ratio = header_ratio
        self.footer_ratio = footer_ratio
        self.criterion = nn.MSELoss()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc = nn.Linear(64 * 16 * 16, 1)

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

    def training_step(self, batch, batch_idx):
        inputs = batch
        outputs = self.forward(inputs)
        loss = self.criterion(outputs, torch.zeros_like(outputs))  # Dummy loss, not used
        return loss

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=0.001)

def train_model(image_paths, max_epochs=10):
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor()
    ])

    dataset = InstagramImageDataset(image_paths, transform=transform)
    dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

    model = ImageCroppingModel()

    trainer = pl.Trainer(max_epochs=max_epochs)
    trainer.fit(model, dataloader)

    return model


def crop_images_with_model(model, image_paths, output_folder, header_ratio=0.17, footer_ratio=0.39):
    cropped_image_paths = []
    for image_path in image_paths:
        try:
            # Load the image
            img = Image.open(image_path)
        except Exception as e:
            print(f"An error occurred when opening the image: {e}")
            return None

        img_width, img_height = img.size

        # Calculate the header and footer height based on the provided ratios
        header_height = int(header_ratio * img_height)
        footer_height = int(footer_ratio * img_height)

        # Calculate the cropping area
        left = 0
        top = header_height
        right = img_width
        bottom = img_height - footer_height

        # Crop the image to remove the header and footer
        cropped_img = img.crop((left, top, right, bottom))

        # Save the cropped image to the output folder
        os.makedirs(output_folder, exist_ok=True)
        cropped_img_path = os.path.join(output_folder, f"{os.path.basename(image_path).rsplit('.', 1)[0]}_cropped.{os.path.basename(image_path).rsplit('.', 1)[-1]}")
        cropped_img.save(cropped_img_path)
        cropped_image_paths.append(cropped_img_path)

    return cropped_image_paths

# Define the path to the folder containing the images
folder_path = '/content/drive/MyDrive/pictures'  # Update this with your actual folder path

# Get a list of all files in the folder
image_files = os.listdir(folder_path)

# Filter out non-image files if needed
image_files = [file for file in image_files if file.endswith(('.jpg', '.jpeg', '.png'))]

# Construct the full paths to the images
image_paths = [os.path.join(folder_path, file) for file in image_files]

# Train the model
trained_model = train_model(image_paths, max_epochs=40)

# Define the output folder path within the mounted Google Drive
output_folder = '/content/drive/MyDrive/pictures/Cropped_Images'

# Crop images using the trained model
cropped_image_paths = crop_images_with_model(trained_model, image_paths, output_folder)

print(f"All cropped images processed. Cropped images: {cropped_image_paths}")




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type       | Params
-----------------------------------------
0 | criterion | MSELoss    | 0     
1 | conv      | Sequential | 23.6 K
2 | fc        | Linear     | 16.4 K
-----------------------------------------
40.0 K    Trainable params
0         Non-trainable params
40.0 K    Total params
0.160     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=40` reached.


All cropped images processed. Cropped images: ['/content/drive/MyDrive/pictures/Cropped_Images/Screenshot_20190926-164658_Instagram_cropped.jpg', '/content/drive/MyDrive/pictures/Cropped_Images/Screenshot_20191012-091619_Instagram_cropped.jpg', '/content/drive/MyDrive/pictures/Cropped_Images/Screenshot_20191111-013222_Instagram_cropped.jpg', '/content/drive/MyDrive/pictures/Cropped_Images/Screenshot_20191005-235421_Instagram_cropped.jpg', '/content/drive/MyDrive/pictures/Cropped_Images/Screenshot_20191007-220028_Instagram_cropped.jpg', '/content/drive/MyDrive/pictures/Cropped_Images/Screenshot_20191126-001343_Instagram_cropped.jpg', '/content/drive/MyDrive/pictures/Cropped_Images/Screenshot_20190926-164722_Instagram_cropped.jpg', '/content/drive/MyDrive/pictures/Cropped_Images/Screenshot_20191005-235439_Instagram_cropped.jpg', '/content/drive/MyDrive/pictures/Cropped_Images/Screenshot_20191006-183430_Instagram_cropped.jpg', '/content/drive/MyDrive/pictures/Cropped_Images/Screenshot_201